Для подтверждения результата необходимо будет прикрепить в форму ноутбук:<br> https://forms.gle/5mJbfaoF2d4ybbkk7. Без прикрепленного ноутбука набранные бонусные баллы начисляться не будут<br>
**Прикрепленный ноутбук должен:**<br>
1) запускаться целиком без ошибок<br>
2) генерировать в результате работы submission.csv<br>
3) все ячейки в ноутбуке должны быть выполнены по порядку (цифры слева от ячеек идут в порядке возрастания, нет ячеек с пустыми скобками)

In [1]:
import regex
import nltk
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Загрузка данных

In [3]:
test_responses = []
with open('hse-sentiment-analysis/x_test.txt',  encoding='utf-8') as file:
    reader = file.read()
    for r in reader.split('\n'):
        test_responses.append(r)

In [4]:
responses = []
with open('hse-sentiment-analysis/x_train.txt',  encoding='utf-8') as file:
    reader = file.read()
    for r in reader.split('\n'):
        responses.append(r)

target = pd.read_csv('hse-sentiment-analysis/y_train.csv')  

In [5]:
print(len(responses))
print(len(test_responses))

3600001
400001


In [6]:
texts = list(map(lambda r: regex.sub(r'[^\p{Latin}]', ' ', r.lower()), responses))
test_texts = list(map(lambda r: regex.sub(r'[^\p{Latin}]', ' ', r.lower()), test_responses))

In [7]:
clean_texts = list(map(lambda r: regex.sub(r' +', ' ', r), texts))
clean_test_texts = list(map(lambda r: regex.sub(r' +', ' ', r), test_texts))

In [8]:
print(len(clean_texts))
print(len(clean_test_texts))

3600001
400001


In [9]:
data = clean_texts[:3600000] + clean_test_texts[:400000]

In [10]:
len(data)

4000000

# Векторизация

In [11]:
vectorizer = TfidfVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(data)

In [12]:
X = vectorizer.transform(data)

In [13]:
Y_train = np.array(target.Probability).astype(int)

In [14]:
len(Y_train)

3600000

In [15]:
lr = LogisticRegression()
lr.fit(X[:3600000], Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
preds = lr.predict_proba(X[3600000: 4000000])[:,1]

In [17]:
submitions = {
  "Id": [i for i in range(1, 400001)],
  "Probability": preds
}

In [18]:
df = pd.DataFrame(submitions)

In [19]:
df

,Id,Probability
0,1,0.992436
1,2,0.995693
2,3,0.035033
3,4,0.667350
4,5,0.986535
...,...,...
399995,399996,0.200603
399996,399997,0.039683
399997,399998,0.121764
399998,399999,0.998612


In [20]:
df.to_csv('submission.csv', index=False)